# Keeping Track of Multiple Choices

Let's say you are tasked to answer $n$ multiple choice questions with $k$ answers each correctly (one correct answer per question). At each attempt, you have to answer all questions. At the end you are told how many questions have been answered correctly, but not which ones. How can this be answered with as few attempts as possible?

I actually faced this problem a few years ago, when I had to take part in some corporate training that suffered from non-sensical content as well as poor translations, making the task more akin to a probabilistic problem than a knowledge test.

First, some formalization. There is a set of questions $Q$, each question $q\in Q$ has multiple answers $a \in A_q$ of which one is correct. We use a collection of binary variables $\mathbf{x}$ for the answers, such that $x_{qa} = 1$ means that answer $a$ is given to question $q$. The variables corresponding to the possible answers to question $q$ are denoted by $\mathbf{x}_q$.

Given an index set $s$ and a collection of binary variables $\mathbf{x}$, we use the sum constraint

$$
C(\mathbf{x}, s, n) = \left\{\begin{matrix}1 & \text{if}\;\sum_{i \in s} x_{i} = n \\ 0 & \text{otherwise}\end{matrix}\right.
$$

Now we can introduce an initial belief state as the (unnormalized) distribution

$$
p_0(\mathbf{x}) = \exp\left(\sum_{q\in Q}\sum_{a \in A_q} \mu_{qa} x_{qa}\right) \, \prod_{q\in Q} C(\mathbf{x}_q, A_q, 1)
$$

Whenever we have answered a set of questions, the response is the number of questions that are correct. Denoting the set of question-answer pairs by $r$ and the response by $n_r$, the outcome is the constraint $C(\mathbf{x}, r, n_r)$. After receiving a number of outcomes $n_r, r \in R$, the belief state is updated like


$$
p(\mathbf{x}) = p_0(\mathbf{x}) \, \prod_{r \in R} C(\mathbf{x}, r, n_r)
$$

where $[\cdots]$ evaluates to one if the expression in the brackets is true, otherwise it is zero. 

This is an exponential family with natural parameter $\mu$, so that marginals can be obtained as

$$
\begin{align}
p(\mathbf{x}) = \nabla_\mu \log Z
\end{align}
$$

where $Z = \sum_{\mathbf{x}} p(\mathbf{x})$ is the partition function. If the initial belief state is uniform, e.g. $\mu_{qa} = 0$, then the marginals are simply given by counting the number of solutions that are compatible with the constraints for each answer and normalizing. For non-uniform $\mu$, we compare the probability masses instead.

When answering questions in sequence, we have to condition on the questions that have already been answered in the current attempt. We denote the set of answered questions by $s$, and use $r \setminus_q s$ to denote all question-answer pairs in $r$ where the question is not in $s$. Then each constraint is updated like

$$
\left[\sum_{q,a \in r} x_{qa} = n_r\right] \Rightarrow \left[\sum_{q, a \in r \setminus_q s} x_{qa} = n_r - |r\cap s|\right]
$$

When the variables in a constraint are all one or zero with certainty, they should be conditioned on as well.

In [23]:
%matplotlib inline
import attr

import numpy as np
import matplotlib.pyplot as plt

In [106]:
@attr.s(slots=True)
class Constraint(object):
    indices = attr.ib(converter=lambda idx: tuple(idx))
    value = attr.ib()
    
    def __init__(self, indices, value):
        self.indices = s

    @property
    def determined(self):
        if self.value == 0:
            return True
        n = len(self.indices[0])
        if self.value == n:
            return True
        return False


@attr.s(slots=True)
class Node(object):
    neighbors = attr.ib()
    data = attr.ib()


class FactorGraph(object):
    def __init__(self):
        self.variables = dict()
        self.factors = dict()
        
    def add_variable(self, key, data=None):
        if key in self.variables:
            raise ValueError()
        self.variables[key] = Node(set(), data)
        
    def add_factor(self, data, variables):
        key = len(self.factors)
        self.factors[key] = Node(set(variables), data)
        for k in variables:
            v = self.variables[k]
            v.neighbors.add(key)
        return key
        
    def __repr__(self):
        return 'FactorGraph({} variables, {} factors)'.format(len(self.variables), len(self.factors))

In [107]:
graph = FactorGraph()

for q in range(nq):
    for a in range(na):
        graph.add_variable((q, a))
        
for q in range(nq):
    idx = [(0, a) for a in range(na)]
    graph.add_factor(Constraint(tuple(zip(*idx)), 1), set(idx))

In [108]:
attempt = random_policy(nq, na)
n = evaluate(solution, random_policy)
answers = [(q, a) for q, a in enumerate(random_policy(nq, na))]
graph.add_factor(Constraint(answers, n), set(answers))

In [111]:
graph.factors

{0: Node(neighbors={(0, 1), (0, 3), (0, 0), (0, 2)}, data=Constraint(indices=((0, 0, 0, 0), (0, 1, 2, 3)), value=1)),
 1: Node(neighbors={(0, 1), (0, 3), (0, 0), (0, 2)}, data=Constraint(indices=((0, 0, 0, 0), (0, 1, 2, 3)), value=1)),
 2: Node(neighbors={(0, 1), (0, 3), (0, 0), (0, 2)}, data=Constraint(indices=((0, 0, 0, 0), (0, 1, 2, 3)), value=1)),
 3: Node(neighbors={(1, 2), (2, 0), (0, 2)}, data=Constraint(indices=((0, 2), (1, 2), (2, 0)), value=0))}

In [12]:
def test_data(nq, na):
    return np.zeros(nq, dtype=int)


def random_policy(nq, na):
    return np.random.choice(na, size=nq, replace=True)


def evaluate(correct, answers):
    return np.sum(correct == answers)

## Constraint propagation

constraint is set of indices and an integer. Constraints and variables form factor graph. Need to

 * reference indices from constraint
 * reference constraints from indices
 
How about using dicts to represent the graph? 

In [21]:
nq, na = 3, 4
solution = test_data(nq, na)
answers = random_policy(nq, na)
score = evaluate(solution, answers)
score

1